In [81]:
# Python black formater for notebook
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

# ETL
We are going to extract multiple data sources from the Dataset folder (csv & json), clean and transform those files into csv format to integrate and process into a Postgrestql database. 

In [82]:
# Importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import glob
import sqlite3

<IPython.core.display.Javascript object>

> First we check if the dataset has null values or maybe there are incompatibilities. By the way, some files are in json and wrongly formatted.

## circuits.csv

In [83]:
circuits_df = pd.read_csv("Datasets/circuits.csv")

<IPython.core.display.Javascript object>

In [84]:
circuits_df.sample(10)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
30,31,donington,Donington Park,Castle Donington,UK,52.8306,-1.37528,88,http://en.wikipedia.org/wiki/Donington_Park
18,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.23470,223,http://en.wikipedia.org/wiki/Indianapolis_Moto...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
50,51,charade,Charade Circuit,Clermont-Ferrand,France,45.7472,3.03889,790,http://en.wikipedia.org/wiki/Charade_Circuit
28,29,adelaide,Adelaide Street Circuit,Adelaide,Australia,-34.9272,138.61700,58,http://en.wikipedia.org/wiki/Adelaide_Street_C...
12,13,spa,Circuit de Spa-Francorchamps,Spa,Belgium,50.4372,5.97139,401,http://en.wikipedia.org/wiki/Circuit_de_Spa-Fr...
29,30,kyalami,Kyalami,Midrand,South Africa,-25.9894,28.07670,1460,http://en.wikipedia.org/wiki/Kyalami
63,64,ain-diab,Ain Diab,Casablanca,Morocco,33.5786,-7.68750,19,http://en.wikipedia.org/wiki/Ain-Diab_Circuit
47,48,mosport,Mosport International Raceway,Ontario,Canada,44.0481,-78.67560,332,http://en.wikipedia.org/wiki/Mosport


<IPython.core.display.Javascript object>

In [85]:
circuits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   77 non-null     int64  
 1   circuitRef  77 non-null     object 
 2   name        77 non-null     object 
 3   location    77 non-null     object 
 4   country     77 non-null     object 
 5   lat         77 non-null     float64
 6   lng         77 non-null     float64
 7   alt         77 non-null     int64  
 8   url         77 non-null     object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.5+ KB


<IPython.core.display.Javascript object>

In [86]:
circuits_df.isnull().sum()

circuitId     0
circuitRef    0
name          0
location      0
country       0
lat           0
lng           0
alt           0
url           0
dtype: int64

<IPython.core.display.Javascript object>

In [87]:
circuits_df.to_csv("Datawarehouse/circuits.csv", index=False)

<IPython.core.display.Javascript object>

## constructors.json

In [88]:
constructors_df = pd.read_json("Datasets/constructors.json", lines=True)

<IPython.core.display.Javascript object>

In [89]:
constructors_df.sample(10)

,constructorId,constructorRef,name,nationality,url
60,62,rebaque,Rebaque,Mexican,http://en.wikipedia.org/wiki/Rebaque
204,207,caterham,Caterham,Malaysian,http://en.wikipedia.org/wiki/Caterham_F1
66,68,lec,LEC,British,http://en.wikipedia.org/wiki/LEC_(Formula_One)
88,90,protos,Protos,British,http://en.wikipedia.org/wiki/Protos_(constructor)
161,163,milano,Milano,Italian,http://en.wikipedia.org/wiki/Scuderia_Milano
175,178,cooper-alfa_romeo,Cooper-Alfa Romeo,British,http://en.wikipedia.org/wiki/Cooper_Car_Company
130,132,lancia,Lancia,Italian,http://en.wikipedia.org/wiki/Lancia_in_Formula...
44,46,onyx,Onyx,British,http://en.wikipedia.org/wiki/Onyx_(racing_team)
59,61,kauhsen,Kauhsen,German,http://en.wikipedia.org/wiki/Kauhsen
116,118,vanwall,Vanwall,British,http://en.wikipedia.org/wiki/Vanwall


<IPython.core.display.Javascript object>

In [90]:
constructors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   211 non-null    int64 
 1   constructorRef  211 non-null    object
 2   name            211 non-null    object
 3   nationality     211 non-null    object
 4   url             211 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.4+ KB


<IPython.core.display.Javascript object>

In [91]:
constructors_df.isnull().sum()

constructorId     0
constructorRef    0
name              0
nationality       0
url               0
dtype: int64

<IPython.core.display.Javascript object>

In [92]:
constructors_df.to_csv("Datawarehouse/constructors.csv", index=False)

<IPython.core.display.Javascript object>

## drivers.json

In [93]:
with open("Datasets/drivers.json") as json_file:
    drivers_data = [json.loads(line) for line in json_file]

drivers_df = pd.json_normalize(drivers_data, sep="_")

<IPython.core.display.Javascript object>

In [94]:
drivers_df["number"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [95]:
drivers_df["code"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [96]:
drivers_df.sample(10)

,driverId,driverRef,number,code,dob,nationality,url,name_forename,name_surname
99,100,comas,NaN,NaN,1963-09-28,French,http://en.wikipedia.org/wiki/%C3%89rik_Comas,Érik,Comas
825,826,kvyat,26.0,KVY,1994-04-26,Russian,http://en.wikipedia.org/wiki/Daniil_Kvyat,Daniil,Kvyat
602,602,la_caze,NaN,NaN,1917-02-26,French,http://en.wikipedia.org/wiki/Robert_La_Caze,Robert,La Caze
53,54,burti,NaN,NaN,1975-03-05,Brazilian,http://en.wikipedia.org/wiki/Luciano_Burti,Luciano,Burti
325,326,wietzes,NaN,NaN,1938-05-28,Canadian,http://en.wikipedia.org/wiki/Eppie_Wietzes,Eppie,Wietzes
56,57,hakkinen,NaN,NaN,1968-09-28,Finnish,http://en.wikipedia.org/wiki/Mika_H%C3%A4kkinen,Mika,Häkkinen
373,374,spence,NaN,NaN,1936-12-30,British,http://en.wikipedia.org/wiki/Mike_Spence,Mike,Spence
320,321,bell,NaN,NaN,1941-10-31,British,http://en.wikipedia.org/wiki/Derek_Bell_(auto_...,Derek,Bell
435,436,campbell-jones,NaN,NaN,1930-01-21,British,http://en.wikipedia.org/wiki/John_Campbell-Jones,John,Campbell-Jones
807,807,hulkenberg,27.0,HUL,1987-08-19,German,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg,Nico,Hülkenberg


<IPython.core.display.Javascript object>

In [97]:
drivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   driverId       853 non-null    int64  
 1   driverRef      853 non-null    object 
 2   number         47 non-null     float64
 3   code           96 non-null     object 
 4   dob            853 non-null    object 
 5   nationality    853 non-null    object 
 6   url            853 non-null    object 
 7   name_forename  853 non-null    object 
 8   name_surname   853 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 60.1+ KB


<IPython.core.display.Javascript object>

In [98]:
drivers_df.isnull().sum()

driverId           0
driverRef          0
number           806
code             757
dob                0
nationality        0
url                0
name_forename      0
name_surname       0
dtype: int64

<IPython.core.display.Javascript object>

In [99]:
drivers_df.to_csv("Datawarehouse/drivers.csv", index=False)

<IPython.core.display.Javascript object>

## pit_stops.json

In [100]:
pit_stops_df = pd.read_json("Datasets/pit_stops.json")

<IPython.core.display.Javascript object>

In [101]:
pit_stops_df.sample(10)

,raceId,driverId,stop,lap,time,duration,milliseconds
5872,977,822,1,41,14:50:11,22.387,22387
7805,1041,825,2,34,15:07:30,23.114,23114
7174,1022,807,1,11,15:37:54,23.475,23475
7302,1026,815,2,41,15:19:44,23.301,23301
6748,1008,828,1,18,15:36:28,23.377,23377
3265,905,825,1,26,14:40:29,28.943,28943
1314,863,18,2,22,15:41:03,22.27,22270
7954,1045,1,3,35,19:36:16,24.345,24345
3123,902,813,1,19,18:37:12,25.395,25395
2039,879,8,3,53,15:21:39,32.309,32309


<IPython.core.display.Javascript object>

In [102]:
pit_stops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8030 entries, 0 to 8029
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        8030 non-null   int64 
 1   driverId      8030 non-null   int64 
 2   stop          8030 non-null   int64 
 3   lap           8030 non-null   int64 
 4   time          8030 non-null   object
 5   duration      8030 non-null   object
 6   milliseconds  8030 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 439.3+ KB


<IPython.core.display.Javascript object>

In [103]:
pit_stops_df["duration"] = pd.to_timedelta(
    pit_stops_df["milliseconds"],
    unit="ms",
)

<IPython.core.display.Javascript object>

In [104]:
pit_stops_df.sample(5)

,raceId,driverId,stop,lap,time,duration,milliseconds
4394,942,834,1,1,14:06:39,0 days 00:00:37.748000,37748
7381,1029,807,1,27,14:47:09,0 days 00:00:23.344000,23344
566,850,4,1,16,14:29:39,0 days 00:00:20.430000,20430
7966,1045,841,3,53,20:07:02,0 days 00:00:24.836000,24836
1351,863,37,3,38,16:10:38,0 days 00:00:24.520000,24520


<IPython.core.display.Javascript object>

In [105]:
pit_stops_df.to_csv("Datawarehouse/pit_stops.csv", index=False)

<IPython.core.display.Javascript object>

## races.csv

In [106]:
races_df = pd.read_csv("Datasets/races.csv")

<IPython.core.display.Javascript object>

In [107]:
races_df.sample(5)

,raceId,year,round,circuitId,name,date,time,url
971,984,2017,16,22,Japanese Grand Prix,2017-10-08,05:00:00,https://en.wikipedia.org/wiki/2017_Japanese_Gr...
902,905,2014,6,6,Monaco Grand Prix,2014-05-25,12:00:00,http://en.wikipedia.org/wiki/2014_Monaco_Grand...
367,368,1989,13,27,Portuguese Grand Prix,1989-09-24,\N,http://en.wikipedia.org/wiki/1989_Portuguese_G...
905,908,2014,9,9,British Grand Prix,2014-07-06,12:00:00,http://en.wikipedia.org/wiki/2014_British_Gran...
285,286,1993,14,27,Portuguese Grand Prix,1993-09-26,\N,http://en.wikipedia.org/wiki/1993_Portuguese_G...


<IPython.core.display.Javascript object>

In [108]:
races_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raceId     1058 non-null   int64 
 1   year       1058 non-null   int64 
 2   round      1058 non-null   int64 
 3   circuitId  1058 non-null   int64 
 4   name       1058 non-null   object
 5   date       1058 non-null   object
 6   time       1058 non-null   object
 7   url        1058 non-null   object
dtypes: int64(4), object(4)
memory usage: 66.2+ KB


<IPython.core.display.Javascript object>

In [109]:
races_df.isnull().sum()

raceId       0
year         0
round        0
circuitId    0
name         0
date         0
time         0
url          0
dtype: int64

<IPython.core.display.Javascript object>

In [110]:
races_df["time"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [111]:
races_df.sample(5)

,raceId,year,round,circuitId,name,date,time,url
420,421,1985,2,27,Portuguese Grand Prix,1985-04-21,NaN,http://en.wikipedia.org/wiki/1985_Portuguese_G...
652,653,1970,11,52,Canadian Grand Prix,1970-09-20,NaN,http://en.wikipedia.org/wiki/1970_Canadian_Gra...
616,617,1973,13,14,Italian Grand Prix,1973-09-09,NaN,http://en.wikipedia.org/wiki/1973_Italian_Gran...
310,311,1991,7,8,French Grand Prix,1991-07-07,NaN,http://en.wikipedia.org/wiki/1991_French_Grand...
382,383,1988,12,14,Italian Grand Prix,1988-09-11,NaN,http://en.wikipedia.org/wiki/1988_Italian_Gran...


<IPython.core.display.Javascript object>

In [112]:
races_df.to_csv("Datawarehouse/races.csv", index=False)

<IPython.core.display.Javascript object>

## results.json

In [113]:
with open("Datasets/results.json") as json_file:
    results_data = [json.loads(line) for line in json_file]

results_df = pd.json_normalize(results_data)

<IPython.core.display.Javascript object>

In [114]:
results_df.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
3391,3392,179,57,1,1,1,1,1,1,10.0,65,1:34:13.665,5653665,\N,\N,\N,\N,1
22297,22300,907,820,206,4,21,17,17,17,0.0,69,\N,\N,40,21,1:14.847,208.072,12
17701,17702,732,341,26,18,5,5,5,5,2.0,51,\N,\N,\N,\N,\N,\N,13
7822,7823,14,8,6,4,12,10,10,10,0.0,61,+58.892,7025229,54,5,1:48.391,168.49,1
23737,23742,987,20,6,5,2,1,1,1,25.0,71,1:31:26.262,5486262,63,7,1:12.539,213.849,1


<IPython.core.display.Javascript object>

In [115]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24960 entries, 0 to 24959
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         24960 non-null  int64  
 1   raceId           24960 non-null  int64  
 2   driverId         24960 non-null  int64  
 3   constructorId    24960 non-null  int64  
 4   number           24960 non-null  object 
 5   grid             24960 non-null  int64  
 6   position         24960 non-null  object 
 7   positionText     24960 non-null  object 
 8   positionOrder    24960 non-null  int64  
 9   points           24960 non-null  float64
 10  laps             24960 non-null  int64  
 11  time             24960 non-null  object 
 12  milliseconds     24960 non-null  object 
 13  fastestLap       24960 non-null  object 
 14  rank             24960 non-null  object 
 15  fastestLapTime   24960 non-null  object 
 16  fastestLapSpeed  24960 non-null  object 
 17  statusId    

<IPython.core.display.Javascript object>

In [116]:
results_df[:].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [117]:
results_df.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
48,49,20,5,1,23,5,5,5,5,4.0,57,+26.789,5493759,49,1,1:33.193,209.062,1
3796,3797,197,74,18,23,21,NaN,R,11,0.0,53,NaN,NaN,NaN,NaN,NaN,NaN,10
11437,11438,479,210,55,33,26,NaN,R,12,0.0,28,NaN,NaN,NaN,NaN,NaN,NaN,20
6426,6427,303,131,17,32,17,7,7,7,0.0,52,NaN,NaN,NaN,NaN,NaN,NaN,11
19687,19688,818,520,113,67,18,NaN,R,28,0.0,65,NaN,NaN,NaN,NaN,NaN,NaN,43


<IPython.core.display.Javascript object>

In [118]:
results_df.isnull().sum()

resultId               0
raceId                 0
driverId               0
constructorId          0
number                 6
grid                   0
position           10735
positionText           0
positionOrder          0
points                 0
laps                   0
time               18368
milliseconds       18369
fastestLap         18410
rank               18249
fastestLapTime     18410
fastestLapSpeed    18410
statusId               0
dtype: int64

<IPython.core.display.Javascript object>

In [119]:
results_df.to_csv("Datawarehouse/results.csv", index=False)

<IPython.core.display.Javascript object>

> I had compressed qualifying files into one and lap_times_split files into a big one.

## qualifying

### qualifying 1

In [120]:
qualifying_split_1_df = pd.read_json("Datasets/Qualifying/qualifying_split_1.json")

<IPython.core.display.Javascript object>

In [121]:
qualifying_split_1_df.sample(5)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
46,47,20,1,1,22,3,1:32.750,1:31.922,1:33.292
1683,1684,192,21,22,5,7,1:18.652,\N,\N
3621,3623,349,4,6,8,10,1:57.023,1:47.544,1:47.441
1967,1968,214,15,20,14,6,1:14.957,\N,\N
1762,1763,200,68,25,21,20,1:34.090,\N,\N


<IPython.core.display.Javascript object>

In [122]:
qualifying_split_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qualifyId      4999 non-null   int64 
 1   raceId         4999 non-null   int64 
 2   driverId       4999 non-null   int64 
 3   constructorId  4999 non-null   int64 
 4   number         4999 non-null   int64 
 5   position       4999 non-null   int64 
 6   q1             4999 non-null   object
 7   q2             4999 non-null   object
 8   q3             4999 non-null   object
dtypes: int64(6), object(3)
memory usage: 351.6+ KB


<IPython.core.display.Javascript object>

In [123]:
qualifying_split_1_df["q2"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [124]:
qualifying_split_1_df["q3"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [125]:
qualifying_split_1_df.sample(5)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
1269,1270,83,2,3,8,12,1:22.086,NaN,NaN
2432,2434,250,44,27,26,9,1:58.021,NaN,NaN
3996,3998,119,52,17,12,18,1:18.341,NaN,NaN
47,48,20,8,6,1,4,1:32.652,1:31.933,1:33.418
189,190,27,13,6,2,2,1:14.921,1:14.747,1:15.859


<IPython.core.display.Javascript object>

### qualifying 2

In [126]:
qualifying_split_2_df = pd.read_json("Datasets/Qualifying/qualifying_split_2.json")

<IPython.core.display.Javascript object>

In [127]:
qualifying_split_2_df.sample(5)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
2327,7349,979,832,5,55,10,1:18.948,1:18.311,1:18.912
2858,7881,1006,20,6,5,2,1:34.569,1:33.079,1:32.298
2929,7952,1009,825,210,20,13,1:37.934,1:37.309,\N
1757,6779,952,815,10,11,9,1:24.698,1:24.003,1:23.782
1828,6850,955,821,210,21,15,1:45.300,1:45.349,\N


<IPython.core.display.Javascript object>

In [128]:
qualifying_split_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3695 entries, 0 to 3694
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qualifyId      3695 non-null   int64 
 1   raceId         3695 non-null   int64 
 2   driverId       3695 non-null   int64 
 3   constructorId  3695 non-null   int64 
 4   number         3695 non-null   int64 
 5   position       3695 non-null   int64 
 6   q1             3695 non-null   object
 7   q2             3695 non-null   object
 8   q3             3695 non-null   object
dtypes: int64(6), object(3)
memory usage: 259.9+ KB


<IPython.core.display.Javascript object>

In [129]:
qualifying_split_2_df["q2"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [130]:
qualifying_split_2_df["q3"].replace("\\N", np.NaN, inplace=True)

<IPython.core.display.Javascript object>

In [131]:
qualifying_split_2_df.isnull().sum()

qualifyId           0
raceId              0
driverId            0
constructorId       0
number              0
position            0
q1                  0
q2                897
q3               1764
dtype: int64

<IPython.core.display.Javascript object>

In [132]:
qualifying_concat = pd.concat(
    [qualifying_split_1_df, qualifying_split_2_df], ignore_index=True
)

<IPython.core.display.Javascript object>

In [133]:
qualifying_concat

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
8689,8730,1047,8,51,7,16,1:37.555,NaN,NaN
8690,8731,1047,825,210,20,17,1:37.863,NaN,NaN
8691,8732,1047,847,3,63,18,1:38.045,NaN,NaN
8692,8733,1047,850,210,51,19,1:38.173,NaN,NaN


<IPython.core.display.Javascript object>

In [134]:
qualifying_concat.to_csv("Datawarehouse/qualifying.csv", index=False)

<IPython.core.display.Javascript object>

## lap_times_split

In [135]:
all_files = glob.glob("Datasets/lap_times/*.csv")
lap_times_df = pd.concat(
    (
        pd.read_csv(
            f,
            header=None,
            names=["raceId", "driverId", "lab", "position", "time", "milliseconds"],
        )
        for f in all_files
    )
)

<IPython.core.display.Javascript object>

In [136]:
lap_times_df.sample(5)

,raceId,driverId,lab,position,time,milliseconds
53520,883,823,5,20,1:43.363,103363
21001,132,30,15,2,1:33.918,93918
82314,1040,825,31,12,1:41.295,101295
4371,149,8,3,9,1:21.871,81871
77311,351,67,12,16,1:56.638,116638


<IPython.core.display.Javascript object>

In [137]:
lap_times_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490904 entries, 0 to 90904
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        490904 non-null  int64 
 1   driverId      490904 non-null  int64 
 2   lab           490904 non-null  int64 
 3   position      490904 non-null  int64 
 4   time          490904 non-null  object
 5   milliseconds  490904 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 26.2+ MB


<IPython.core.display.Javascript object>

In [138]:
lap_times_df.to_csv("Datawarehouse/lap_times.csv", index=False)

<IPython.core.display.Javascript object>

> After cleaning, write new CSV files for each dataframe constructor, then ingest all CSV fdata by using pandas to_sql if the file exists, lest replace it and set the index to false.

## Ingest data

In [139]:
conn = sqlite3.connect("sql_app.db")

<IPython.core.display.Javascript object>

In [140]:
circuit_df = pd.read_csv("Datawarehouse/circuits.csv")
circuit_df.to_sql("circuit", conn, if_exists="replace", index=False)

77

<IPython.core.display.Javascript object>

In [141]:
constructor_df = pd.read_csv("Datawarehouse/constructors.csv")
constructor_df.to_sql("constructor", conn, if_exists="replace", index=False)

211

<IPython.core.display.Javascript object>

In [142]:
driver_df = pd.read_csv("Datawarehouse/drivers.csv")
driver_df.to_sql("driver", conn, if_exists="replace", index=False)

853

<IPython.core.display.Javascript object>

In [143]:
pit_stop_df = pd.read_csv("Datawarehouse/pit_stops.csv")
pit_stop_df.to_sql("pitstop", conn, if_exists="replace", index=False)

8030

<IPython.core.display.Javascript object>

In [144]:
race_df = pd.read_csv("Datawarehouse/races.csv")
race_df.to_sql("race", conn, if_exists="replace", index=False)

1058

<IPython.core.display.Javascript object>

In [145]:
result_df = pd.read_csv("Datawarehouse/results.csv")
result_df.to_sql("result", conn, if_exists="replace", index=False)

24960

<IPython.core.display.Javascript object>

In [146]:
qualifying_df = pd.read_csv("Datawarehouse/qualifying.csv")
qualifying_df.to_sql("qualifying", conn, if_exists="replace", index=False)

8694

<IPython.core.display.Javascript object>

In [147]:
lab_time_df = pd.read_csv("Datawarehouse/lap_times.csv")
lab_time_df.to_sql("labtimessplit", conn, if_exists="replace", index=False)

490904

<IPython.core.display.Javascript object>